In [1]:
import numpy as np
from matplotlib import pyplot as plt
import py_gauge_mc
import os
from tqdm import tqdm

In [2]:
def make_graph(l, vns, inv_ks):
    pots = np.einsum('i,j->ij', inv_ks, vns)
    graph = py_gauge_mc.GPUGaugeTheory(l,l,l,l,pots.astype(np.float32))
    return graph, pots

In [3]:
L = 8
vlen = 128
n_replicas = 128 
inv_k = 0.1

inv_ks = [inv_k]*n_replicas
vns = [(n**2) for n in range(vlen)]

graph,vn = make_graph(L,vns,inv_ks)

In [ ]:
start = 0.1
end = 1.0
ndivs = 128
inc = np.power(end/start,1.0/ndivs)

graph.simulate(10000,run_parallel_tempering=False,run_global_updates=True)
windings, energies = graph.simulate_and_get_winding_nums_and_energies(1000,steps_per_sample=10, run_parallel_tempering=False,run_global_updates=True) 

all_inv_ks = [inv_k]
all_energies = [energies]
all_windings = [windings]
for i in tqdm(range(ndivs)):
    graph.scale_potentials_by_factor(inc)
    inv_k = inv_k * inc
    graph.simulate(10000,run_parallel_tempering=False,run_global_updates=True)
    windings, energies = graph.simulate_and_get_winding_nums_and_energies(1000,steps_per_sample=10, run_parallel_tempering=False,run_global_updates=True)
    all_energies.append(energies)
    all_windings.append(windings)
    all_inv_ks.append(inv_k)
    
all_inv_ks = np.array(all_inv_ks)
all_energies = np.array(all_energies)
all_windings = np.array(all_windings)

  1%|▊                                                                                                       | 1/128 [00:22<48:06, 22.73s/it]

In [ ]:
plt.plot(all_inv_ks, all_energies.mean(axis=(-1,-2))/L**4)
plt.show()

plt.plot(all_inv_ks, all_energies.var(axis=(-1,-2))/L**4)
plt.show()

In [ ]:
plt.plot(all_inv_ks, all_windings.var(axis=(-1,-2,-3)))
plt.show()